# Linear Regression Consulting Project

You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

**Imports**

In [63]:
import findspark
findspark.init('/home/sedat/spark-3.3.2-bin-hadoop3')
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

**Start session and read csv file**

In [64]:
spark = SparkSession.builder.appName('ships_lr').getOrCreate()

In [65]:
df = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [66]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [67]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



**Total data**

In [68]:
df.count()

158

**Column names**

In [69]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

**First Data**

In [70]:
df.head(1)[0]

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)

In [71]:
for index, item in enumerate(df.head(1)[0]):
    print(df.columns[index] + ':', item)

Ship_name: Journey
Cruise_line: Azamara
Age: 6
Tonnage: 30.276999999999997
passengers: 6.94
length: 5.94
cabins: 3.55
passenger_density: 42.64
crew: 3.55


**Crusie_line is so important feature so we will convert to as a numerical value**

In [72]:
df.groupby('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [73]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_num")

In [74]:
df = indexer.fit(df).transform(df)
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_num|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|            1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|            1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|            1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|

**Vector Assembler**

In [75]:
assembler = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers', 
                                       'length', 'cabins', 'passenger_density',
                                       'Cruise_line_num'],
                           outputCol='features')

In [76]:
output = assembler.transform(df)

In [78]:
output.head(1)[0]

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_num=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]))

**Final model data**

In [79]:
final_data = output.select('features', 'crew')

In [80]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



**Split data (train, and test)**

In [81]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [90]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              114|
|   mean|7.980087719298257|
| stddev|3.642601544217585|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [91]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                44|
|   mean| 7.312500000000001|
| stddev|3.1013471379011155|
|    min|               1.6|
|    max|              13.6|
+-------+------------------+



**Model**

In [96]:
lr = LinearRegression(labelCol='crew')
lr_model = lr.fit(train_data)

23/03/19 16:45:00 WARN Instrumentation: [92609980] regParam is zero, which might cause numerical instability and overfitting.


In [105]:
lr_model.coefficients

DenseVector([-0.0151, 0.0094, -0.1859, 0.4227, 0.9452, -0.0078, 0.058])

In [106]:
lr_model.intercept

-0.975436377131388

**Test model. Check residuals and error metrics**

In [97]:
test_residuals = lr_model.evaluate(test_data)

In [99]:
test_residuals.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  0.9674591150189853|
| -1.6243565460651777|
| -0.8393271218280685|
|   0.510065850798318|
| -0.6507243245328898|
| -0.7169352377527645|
|  0.9596658327914511|
|-0.21781296369301728|
|  1.8471659316700597|
| 0.15232080249039903|
| -0.5616162626647068|
|-0.26675753412440706|
|-0.20528115434233918|
|0.021247565082544995|
| 0.07391795883641539|
| -0.4794806816231709|
|  0.7470158968820204|
|  0.7808855082471649|
|  0.1478569204064808|
|  -1.180274820302305|
+--------------------+
only showing top 20 rows



In [107]:
print(f'RMSE: {test_residuals.rootMeanSquaredError}')
print(f'MSE: {test_residuals.meanSquaredError}')
print(f'MAE: {test_residuals.meanAbsoluteError}')

RMSE: 0.6718319607482413
MSE: 0.4513581834828264
MAE: 0.5130927687942022


In [101]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                44|
|   mean| 7.312500000000001|
| stddev|3.1013471379011155|
|    min|               1.6|
|    max|              13.6|
+-------+------------------+

